<a href="https://colab.research.google.com/github/aim56009/AI-MA_project/blob/main/Chew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle
import torchvision.transforms.functional as TF 

from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import glob
import os

import random
import click
import sklearn
import sklearn.model_selection
from collections import defaultdict
from sklearn.metrics import accuracy_score
from pathlib import Path
import sys
from torch import optim
from torch.optim import lr_scheduler
from functools import partial
import matplotlib.pyplot as plt

In [ ]:
!git clone https://github.com/aim56009/AI-MA_project.git

# Chew


In [ ]:
!pip3 install partitura
import partitura
import numpy as np
import os
import pickle
from sklearn.metrics import accuracy_score

In [81]:
def evaluation_chew():

    history = defaultdict(list)
    path = "AI-MA_project/chorales_converted"

    ### load the chorales in xml format ###
    for filename in os.listdir(path):
        if not filename.endswith('.xml'): continue
        fullname = os.path.join(path, filename)
        

        part = partitura.load_musicxml(fullname)
            

        ### apply chews method ### 
        chew_sep = partitura.musicanalysis.estimate_voices(part, monophonic_voices=True)


        ### seperate the results -> e.g. pos_zero = all positions where chew prediction says voice 0 ####
        pos_zero = np.where(chew_sep==1)
        pos_one = np.where(chew_sep==2)
        pos_two = np.where(chew_sep==3)
        pos_three = np.where(chew_sep==4)


        ### create notearray object that contain only the corresponding voice ###
        note_array_zero = partitura.utils.ensure_notearray(part)[pos_zero]
        note_array_one = partitura.utils.ensure_notearray(part)[pos_one]
        note_array_two = partitura.utils.ensure_notearray(part)[pos_two]
        note_array_three = partitura.utils.ensure_notearray(part)[pos_three]


        ### create pr representation of all voices ###
        pr_zero = partitura.utils.compute_pianoroll(note_array_zero,time_unit = "beat",time_div = 12)
        pr_zero = pr_zero.toarray()

        pr_one = partitura.utils.compute_pianoroll(note_array_one,time_unit = "beat",time_div = 12)
        pr_one = pr_one.toarray()

        pr_two = partitura.utils.compute_pianoroll(note_array_two,time_unit = "beat",time_div = 12)
        pr_two = pr_two.toarray()

        pr_three = partitura.utils.compute_pianoroll(note_array_three,time_unit = "beat",time_div = 12)
        pr_three = pr_three.toarray()


        ### get original data (as labels) ###
        part_zero = part[0]
        part_one = part[1]
        part_two = part[2]
        part_three = part[3]


        ### compute pianoroll representation of original data for computing acc ###
        pr_zero_ori = partitura.utils.compute_pianoroll(part_zero, time_unit = "beat",time_div = 12)
        v0 = pr_zero_ori.toarray()

        pr_one_ori = partitura.utils.compute_pianoroll(part_one, time_unit = "beat",time_div = 12)
        v1 = pr_one_ori.toarray()

        pr_two_ori = partitura.utils.compute_pianoroll(part_two, time_unit = "beat",time_div = 12)
        v2 = pr_two_ori.toarray()

        pr_three_ori = partitura.utils.compute_pianoroll(part_three, time_unit = "beat",time_div = 12)
        v3 = pr_three_ori.toarray()

        pr_ori = partitura.utils.compute_pianoroll(part, time_unit = "beat",time_div = 12)
        v_all = pr_ori.toarray()

        if pr_zero.shape == pr_one.shape == pr_two.shape == pr_three.shape  :
            ### compute accuracy ###
            acc_v0 = accuracy_score(torch.tensor(pr_zero), v0)
            acc_v1 = accuracy_score(torch.tensor(pr_one), v1)
            acc_v2 = accuracy_score(torch.tensor(pr_two),v2)
            acc_v3 = accuracy_score(torch.tensor(pr_three), v3)

            history["accuracy_v0"].append(acc_v0)
            history["accuracy_v1"].append(acc_v1)
            history["accuracy_v2"].append(acc_v2)
            history["accuracy_v3"].append(acc_v3)
            
        else: print("pr shape does not match for:", filename)
        
    return history

In [82]:
hist_acc =  evaluation_chew()

pr shape does not match for: chor245.xml
pr shape does not match for: chor264.xml
pr shape does not match for: chor286.xml
pr shape does not match for: chor056.xml
pr shape does not match for: chor283.xml
pr shape does not match for: chor074.xml
pr shape does not match for: chor043.xml
pr shape does not match for: chor252.xml
pr shape does not match for: chor133.xml
pr shape does not match for: chor095.xml


In [83]:
total_len = len(hist_acc["accuracy_v0"])
total_len

360

In [84]:
accuracy_v0 = np.sum(hist_acc["accuracy_v0"])/total_len
accuracy_v1 = np.sum(hist_acc["accuracy_v1"])/total_len
accuracy_v2 = np.sum(hist_acc["accuracy_v2"])/total_len
accuracy_v3 = np.sum(hist_acc["accuracy_v3"])/total_len

print("accuracy of v0-v3:",accuracy_v0,accuracy_v1,accuracy_v2,accuracy_v3)

accuracy of v0-v3: 0.9882161458333333 0.9801215277777777 0.9844618055555555 0.9959201388888889
